In [29]:
import ross as rs
import numpy as np
import plotly.graph_objects as go

Q_ = rs.Q_
rs.__version__

'1.5.0'

Just claculate the first rotor, need to have this too take the matrice, that we going to use

In [30]:
material_exo = rs.Material(name="mat_exo", rho=7850, E=2.05 * 10**11, Poisson=0.3)


Low_pressure_elements = []
low_pressure_seal_elements = []
low_pressure_elements = []

for i in range(8) : 
    Low_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0, odl=0.05))

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257)
)

low_pressure_elements.append(
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3)
)

low_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=8, kxx=3e7, kyy=3e7, cxx=0, cyy=0))
low_pressure_seal_elements.append(rs.BearingElement(n=6, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0))

rotor_lp = rs.Rotor(
    shaft_elements=Low_pressure_elements,
    bearing_elements=low_pressure_seal_elements,
    disk_elements=low_pressure_elements,
)

rotor_lp.plot_rotor()

In [31]:
samples = 40
max_spin = 20000/60 * 2 * np.pi
speed_range = np.linspace(0, max_spin, samples)
campbell = rotor_lp.run_campbell(speed_range, frequency_type="wn", frequencies=7)
fig = campbell.plot(frequency_units="rad/s", width=600, height=600)
# for i in fig.data:
#     try:
#         i["y"] = i["y"] / 60  * 2 * np.pi # pass in hz   
#     except:
#         pass
fig.update_yaxes(title="Natural Frequencies (Hz)", range=[0, 90])
fig.update_xaxes(title="Rotor Spin Speed (rpm)", range=[0, max_spin * 60 / (2 * np.pi)])

In [32]:
hight_pressure_elements = []
hight_pressure_seal_elements = []
hight_pressure_disk_elements = []

for i in range(4) : 
    hight_pressure_elements.append(rs.ShaftElement(L = 0.1, material=material_exo, n=i, idl=0.07, odl=0.08))

hight_pressure_disk_elements.append(
    rs.DiskElement.from_geometry(n=2, material=material_exo, width=0.015, i_d=0.08, o_d=0.038)
)

hight_pressure_seal_elements.append(rs.BearingElement(n=0, kxx=2.5e7, kyy=2.5e7, cxx=0, cyy=0))
hight_pressure_seal_elements.append(rs.BearingElement(n=4, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0))
rotor_hp = rs.Rotor(
    shaft_elements=hight_pressure_elements,
    bearing_elements=hight_pressure_seal_elements,
    disk_elements=hight_pressure_disk_elements,
)

rotor_hp.plot_rotor()


In [33]:
campbell = rotor_hp.run_campbell(speed_range, frequency_type="wn", frequencies=7)
fig = campbell.plot(frequency_units="rad/s", width=600, height=600)
fig

In [34]:
modal = rotor_hp.run_modal(53.14, 7)
modal.wd[:1]

array([3637.56286722])

Need to create one birotor with two shft two extract the therm K and M, which didn't depend on the gyroscopique effect. And so on the RPM

In [35]:

axial_shaft = [rs.ShaftElement(0.1, 0, 0.05, material=material_exo) for _ in range(8)]


coaxial_shaft = [rs.ShaftElement(0.1, 0.07, 0.08, material=material_exo) for _ in range(4)]


disks = [
    rs.DiskElement.from_geometry(n=1, material=material_exo, width=0.02, i_d=0.05, o_d=0.257),
    rs.DiskElement.from_geometry(n=7, material=material_exo, width=0.02, i_d=0.05, o_d=0.3),
    rs.DiskElement.from_geometry(n=11, material=material_exo, width=0.015, i_d=0.08, o_d=0.038)
]

bearings = [
    rs.BearingElement(0, kxx=3*10**7, kyy=3*10**7, cxx=0),
    rs.BearingElement(8, kxx=3*10**7, kyy=3*10**7, cxx=0),
    rs.BearingElement(9, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0),
    rs.BearingElement(6, n_link=13, kxx=2.5*10**7, kyy=2.5*10**7, cxx=0)
]
""
# Création du rotor coaxial
shaft = [axial_shaft, coaxial_shaft]
rotor_coaxial = rs.CoAxialRotor(shaft, disks, bearings)
rotor_coaxial.plot_rotor()

In [36]:
# modal = rotor_coaxial.run_modal(4,)
# fig = modal.plot_mode_3d(0, frequency_units="rpm", width=600, height=600)
# fig.layout.title.text = "FIRST CRITICAL SPEED" + fig.layout.title.text
# # fig.show()

In [37]:
modal = rotor_coaxial.run_modal(speed=Q_(0, "RPM"))
fig = modal.plot_mode_3d(1, frequency_units="rpm", width=600, height=600)
fig.layout.title.text = "SECOND CRITICAL SPEED" + fig.layout.title.text
fig.show()

Extract the diverse matrix

In [38]:
G_lp = rotor_lp.G()
print("Gyrioscopique Low pressure \t",G_lp.shape)
G_hp = rotor_hp.G() * 1.5
print("Gyrioscopique High pressure \t",G_hp.shape)

# campbell = rotor_coaxial.run_campbell(speed_range=Q_(list(range(0, 4500, 50)), "RPM"))


Gyrioscopique Low pressure 	 (36, 36)
Gyrioscopique High pressure 	 (20, 20)


In [39]:
G1 = np.hstack((G_lp, np.zeros((G_lp.shape[0], rotor_coaxial.ndof - G_lp.shape[1]))))
G2 = np.hstack((np.zeros((G_hp.shape[0],rotor_coaxial.ndof - G_hp.shape[1])), G_hp))

G = np.vstack((G1, G2))

In [40]:
import complete_ross as fct
max_spin = 20000/60 * 2 * np.pi
samples = 40
speed_range = np.linspace(0, max_spin, samples)
fct.run_campbell(rotor_coaxial, speed_range, frequencies = 7, Gyro=G) 

In [41]:
fct.run_cambell_2_rotor(rotor_lp, rotor_hp, speed_range, frequencies = 7)

In [42]:
fct.run_campbell(rotor_lp, speed_range, frequencies = 7)

In [43]:
fct.run_campbell(rotor_hp, speed_range, frequencies = 7, Gyro=G_hp*1.5, slope_critic_speed=1.5)

In [44]:
fct.run_damping_mode(rotor_lp, rotor_hp, speed_range, frequencies = 7)